In [90]:
import pyspark
from pyspark.mllib.regression import LabeledPoint as DataPoint


In [88]:
RATINGS_10M = './ml-10M100K/ratings.dat'
def parseData(row):
    data = row.split("::")
    return DataPoint(data[2], data[:2] + data[3:])

rdd = sc.textFile(RATINGS_10M).map(parseData).sortBy(keyfunc = lambda x: x.features[2], ascending = False)



In [94]:
size = rdd.count()
trainPartVal = rdd.zipWithIndex().filter(lambda x: x[-1] < size*0.6)
print trainPartVal.count()

#train = rdd.filter(lambda x: x<= )

[(LabeledPoint(3.0, [62510.0,34148.0,1231131736.0]), 0),
 (LabeledPoint(2.5, [62510.0,4784.0,1231131303.0]), 1),
 (LabeledPoint(2.0, [63100.0,2953.0,1231131142.0]), 2),
 (LabeledPoint(3.5, [63100.0,7000.0,1231131137.0]), 3),
 (LabeledPoint(3.5, [63100.0,2478.0,1231131132.0]), 4)]

In [95]:
print size

10000054


In [87]:
train.take(5)

Exception: It appears that you are attempting to broadcast an RDD or reference an RDD from an action or transformation. RDD transformations and actions can only be invoked by the driver, not inside of other transformations; for example, rdd1.map(lambda x: rdd2.values.count() * x) is invalid because the values transformation and count action cannot be performed inside of the rdd1.map transformation. For more information, see SPARK-5063.

[LabeledPoint(3.0, [62510.0,34148.0,1231131736.0]),
 LabeledPoint(2.5, [62510.0,4784.0,1231131303.0]),
 LabeledPoint(4.5, [63100.0,2361.0,1231131107.0]),
 LabeledPoint(4.0, [63100.0,4678.0,1231131098.0]),
 LabeledPoint(4.5, [63100.0,2657.0,1231131088.0])]

In [85]:
rdd.take(5)

[LabeledPoint(3.0, [62510.0,34148.0,1231131736.0]),
 LabeledPoint(2.5, [62510.0,4784.0,1231131303.0]),
 LabeledPoint(2.0, [63100.0,2953.0,1231131142.0]),
 LabeledPoint(3.5, [63100.0,7000.0,1231131137.0]),
 LabeledPoint(3.5, [63100.0,2478.0,1231131132.0])]